# FastAPI based application for Diabetes Model

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model as keras_load_mdel
from fastapi import FastAPI, UploadFile, File

In [2]:
def load_model():
    """
    Loads the pre_trained keras CNN model
    """
    model_path = "model/diabetes_model.h5"
    model = keras_load_model(model_path)
    return model

In [3]:
def preprocessing_data(df: pd.DataFrame):
    """
    Preprocess the Excel input data to match the CNN expected input shape
    """
    df = df.dropna() # Drops rows with missing values
    data = df.values # converts dataframes into numpy arrays
    data = data.astype("float32")# ensures array is of float32 typ
    #reshaping data for 1D CNN
    if data.ndim == 2:
        data = np.expand_dims(data, axis = -1)
    return data

In [4]:
def predict(model, data):
    """
    predicting using CNN loaded model
    """
    pred_probs = model.predict(data) #getting predictions
    predictions = (pred_probs > 0.5).astype("int32") #converting probabilities into binary outcomes
    predictions = np.squeeze(predictions) #if output shape has an extra dimension, squeeze it.
    return predictions

In [5]:
app = FastAPI(title = "Diabetes Model")
model = load_model

@app.post("/predict")
async def predict_diabetes(file: UploadFile = File(...)):
    df = pandas.read_excel(file.file)
    data = preprocess_data(df)
    preds = predict(model, data)
    return {"predictions": pres.tolist()}